In [ ]:
import requests
import urllib.parse
import json
import os

# === Step 1: Get Cognito OAuth2 Token ===
COGNITO_TOKEN_URL = "https://agentcore-<your-name>.auth.us-east-1.amazoncognito.com/oauth2/token"  # Replace with your Cognito Token URL
CLIENT_ID = "your-client-id"  # Replace with your Client ID
CLIENT_SECRET = "your-client-secret"  # Replace with your Client Secret
SCOPE = "default-m2m-resource-server-dotx2k/read"

token_response = requests.post(
    COGNITO_TOKEN_URL,
    data=f"grant_type=client_credentials&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&scope={SCOPE}",
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)
token_json = token_response.json()
access_token = token_json.get("access_token")
if not access_token:
    print("Failed to get access token:", token_json)
    exit(1)

print("✅ Got Cognito access token.")
print(access_token)

In [ ]:
# === Step 2: Invoke AgentCore Runtime ===
invoke_agent_arn = "arn:aws:bedrock-agentcore:us-east-1:YOUR_ACCOUNT_ID:runtime/your-agent-name"  # Replace with your Agent ARN
REGION_NAME = "us-east-1"
escaped_agent_arn = urllib.parse.quote(invoke_agent_arn, safe='')
AGENTCORE_ENDPOINT = f"https://bedrock-agentcore.{REGION_NAME}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

payload = {
    "weburl": "https://example.com",  # Replace with your website URL
    "username": "your-username",  # Replace with username
    "password": "your-password",  # Replace with password
    "promptfile": "Your task instructions here",  # Replace with task instructions
    "client_name": "your-client-name"  # Replace with client name
}

# Enable verbose logging for requests
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger("urllib3.connectionpool").setLevel(logging.DEBUG)

invoke_response = requests.post(
    AGENTCORE_ENDPOINT,
    headers=headers,
    data=json.dumps(payload)
)

# Print response in a safe manner
print(f"Status Code: {invoke_response.status_code}")
print(f"Response Headers: {dict(invoke_response.headers)}")

# Handle response based on status code
if invoke_response.status_code == 200:
    response_data = invoke_response.json()
    print("Response JSON:")
    print(json.dumps(response_data, indent=2))  
elif invoke_response.status_code >= 400:
    print(f"Error Response ({invoke_response.status_code}):")
    error_data = invoke_response.json()
    print(json.dumps(error_data, indent=2))
    
else:
    print(f"Unexpected status code: {invoke_response.status_code}")
    print("Response text:")
    print(invoke_response.text[:500])